# TO DO:
- Make encoder not encode values that are already numerical
- Create not slave logs training set??
- Figure out how to install rpy2 so that we can ad the second data set...
- Mess with classifier parameters
- test classifier

In [1]:
import numpy as np
import pandas as pd
import exploringShipLogbooks.wordCount as wc
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB

import collections

#import rpy2
#from rpy2.robjects import pandas2ri
#pandas2ri.activate()

from exploringShipLogbooks.basic_utils import extract_logbook_data
from exploringShipLogbooks.basic_utils import isolate_columns
#from exploringShipLogbooks.basic_utils import MultiColumnLabelEncoder
from exploringShipLogbooks.basic_utils import encode_data_df
from exploringShipLogbooks.basic_utils import clean_data

from exploringShipLogbooks.config import *


# Load and clean data
### Load CLIWOC ship logs

In [2]:
# extract data from zip file
cliwoc_data = extract_logbook_data('CLIWOC15.csv')
#cliwoc_data = cliwoc_data.loc[:100000]
#len(cliwoc_data)

/Applications/miniconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (5,6,7,8,11,13,18,19,23,24,25,26,28,29,30,34,35,38,43,44,46,73,77,81,82,84,85,87,88,94,96,97,98,99,111,114,116,119,120,122,124,125,127,129,131,133,135,137,140) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


### Find slave training data in CLIWOC data set

In [3]:
# extract logs that mention slaves
mentions_slaves = wc.count_key_words(cliwoc_data, text_columns, slave_words)
slave_index = mentions_slaves[(mentions_slaves['ContainsKeyWord'] != 0)].index
print('Found ', len(slave_index), ' logs that mention slaves')

# cleanup
del mentions_slaves

Found  464  logs that mention slaves


### Clean CLIWOC data

In [ ]:
# remove undesired columns
cliwoc_data = isolate_columns(cliwoc_data, desired_columns)

# clean data (make all same case)
cliwoc_data = clean_data(cliwoc_data)

In [5]:
# I know you don't like using how I set this up previously, but it runs much faster for me since it only deals with 
# list and not the df directly

from exploringShipLogbooks.basic_utils import remove_undesired_columns
undesired_columns = remove_undesired_columns(cliwoc_data, desired_columns)
cliwoc_data = cliwoc_data.drop(undesired_columns, axis=1)
cliwoc_data = clean_data(cliwoc_data)

### Load Slave Voyages data

In [ ]:
#filename = './exploringShipLogbooks/data/tastdb-exp-2010.sav'
#slave_voyage_logs = rpy2.robjects.r('foreign::read.spss("%s", to.data.frame=TRUE)' % filename)



### Clean Slave voyages data

In [ ]:
#slave_voyage_desired_cols = ['portdep', 'portret', 'shipname', 'rig', 'national', 'yeardep']
#slave_voyage_logs = isolate_columns(slave_voyage_logs, slave_voyage_desired_cols)

#slave_voyage_logs.columns = ['ShipName', 'Nationality', 'ShipType', 'VoyageFrom', 'VoyageTo', 'Year']

#slave_voyage_logs = clean_data(slave_voyage_logs)

# Add indices to slave indices!

# Join data sets

In [6]:
all_data = cliwoc_data # .append(slave_voyage_logs, ignore_index = True)

del cliwoc_data

# Encode data
- Must encode data before separating, otherwise values that do not occur in a subset will be encoded differently

In [7]:
# TODO: should change encoder to only encode categorical columns!
# we should probably split the data and then encode it (if we care about speed). although this runs on my computer 
# in ~30 seconds. was this the line that wasn't working for you?

all_data = encode_data_df(all_data)

# Extract training data, and create list of classes

In [8]:
len(all_data)

280280

In [9]:
# split log data into data that mentions slaves and data that does not
# ** will add new data set to this once I can use R!
slave_logs = all_data.loc[slave_index.values]

# ****arbitrarily using first 1000 logs as "no slave" training data.
# need to figure out what to put here.
no_slave_index = range(1000)
no_slave_logs = all_data.loc[no_slave_index]

# remaining data is unclassified 
unclassified_logs = all_data.drop(slave_index)
unclassified_logs = all_data.drop(no_slave_index)

# convert to numpy array
unclassified_logs = unclassified_logs.as_matrix()

# clean-up
del all_data

In [10]:
# create list of classes for training data (0 is for non-slave, 1 is for slave)
# index matches training_data
classes = np.zeros(len(no_slave_logs))
classes = np.append(classes, np.ones(len(slave_logs)))

# joint training data
training_data = no_slave_logs.append(slave_logs, ignore_index=True)

# convert to numpy array
training_data = training_data.as_matrix()

# cleanup
del no_slave_logs, slave_logs

# Fit training data to classifier

In [11]:
classifier = MultinomialNB(alpha = 1.0, class_prior = None, fit_prior = True)
classifier.fit(training_data, classes)

predictions = classifier.predict(unclassified_logs)



In [12]:
predictions_count = collections.Counter(predictions)
print(predictions_count)

Counter({1.0: 219108, 0.0: 60172})
